In [ ]:
!pip install pymp-pypi


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pymp-pypi: filename=pymp_pypi-0.5.0-py3-none-any.whl size=10339 sha256=be3a42a4ba259d1b0ebbf206f3517e0d557327e92a37d93ae1cc85fdbf0949a6
  Stored in directory: /root/.cache/pip/wheels/98/1d/32/d3b1174e02ae373cde389cd6fccc24c8f2530509b138282702
Successfully built pymp-pypi


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pymp

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving web-Google.txt.gz to web-Google.txt.gz


In [ ]:
import gzip
import shutil
with gzip.open('/content/web-Google.txt.gz', 'rb') as f_in:
    with open('file.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
import networkx as nx

G_tmp = nx.read_edgelist('/content/file.txt', create_using = nx.DiGraph)
print(nx.info(G_tmp))

DiGraph with 875713 nodes and 5105039 edges


In [ ]:
c = sorted(nx.weakly_connected_components(G_tmp), key=len, reverse=True)
wcc_set = c[0]
G = G_tmp.subgraph(wcc_set)
print(nx.info(G))

DiGraph with 855802 nodes and 5066842 edges


In [ ]:
tele_p = 0.15
A = nx.adjacency_matrix(G)
A = A.T

n = A.shape[0]
A = A.tocsr()
print(type(A))
A = A.astype('float64')

# A = np.array(A, dtype = 'float64')

out_degrees = np.array([G.out_degree[x] for x in list(G.nodes)], dtype='float64')
A.data *= (1-tele_p) / np.take(out_degrees, A.indices)
# A.data /= np.take(out_degrees, A.indices)

<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
%%time
r =  np.ones((n,1))
error = 1
for _ in range(50):
#     if _ % 100 == 0:
#         print("iter", _)
    r_new = A @ r
    r_new += np.sum(r) * tele_p / n
#     r_new /= np.sum(r_new)
    error = np.sum(np.absolute(r - r_new))
    r = r_new
    if error < 1e-6:
        break

r /= np.sum(r)
r = r.reshape(n)


print("part of my result\n", r[:50])

part of my result
 [3.64705701e-05 3.79228355e-05 8.65459111e-06 3.84303158e-05
 3.71368858e-05 7.18387504e-06 1.08754748e-05 3.38817839e-06
 3.93632510e-06 3.93632510e-06 4.89127502e-06 2.72410296e-06
 2.84383173e-06 3.03017014e-06 8.03988628e-06 3.36158727e-06
 1.03354893e-06 1.40558986e-04 3.27626883e-04 7.48129577e-06
 7.28870966e-06 1.03969881e-06 4.90884656e-06 4.59075507e-06
 3.58446230e-06 6.74313229e-06 3.80842318e-06 3.32648748e-06
 1.49044403e-05 3.55707200e-06 3.52658037e-06 3.52658037e-06
 3.52658037e-06 4.52032223e-06 2.30537555e-06 9.70917221e-06
 1.22010316e-05 3.31036020e-04 9.34368558e-06 9.22117384e-06
 9.50054616e-06 9.58088277e-06 9.55104543e-06 9.39608572e-06
 9.44656859e-06 1.88049081e-06 1.87682800e-06 2.00320673e-06
 5.09908899e-06 1.87682800e-06]
CPU times: user 1.02 s, sys: 13.7 ms, total: 1.03 s
Wall time: 1.02 s


In [ ]:
truth = nx.pagerank(G, alpha=0.85, max_iter = 100)
truth = np.array(list(truth.values()), dtype='float64')
print("part of networkx's result\n",truth[:50])

part of networkx's result
 [2.83545918e-05 3.17140006e-05 7.15313456e-06 3.18433656e-05
 3.10020184e-05 5.45502590e-06 1.15575220e-05 3.07694855e-06
 2.69228185e-06 2.69228185e-06 5.17349351e-06 2.05934593e-06
 2.08156160e-06 2.34779737e-06 6.11384378e-06 2.47537605e-06
 5.87459196e-07 1.75093821e-04 3.71590743e-04 7.08537651e-06
 3.70897411e-06 5.91671476e-07 3.97260442e-06 3.61127278e-06
 2.75852433e-06 5.65562148e-06 2.81977746e-06 2.43426924e-06
 1.10552532e-05 2.81129815e-06 2.76713990e-06 2.76713990e-06
 2.76713990e-06 4.60586095e-06 1.94053971e-06 7.50825438e-06
 8.07304360e-06 3.13681305e-04 7.21791541e-06 7.11620490e-06
 7.40872488e-06 7.88138455e-06 7.44274097e-06 7.26072719e-06
 7.49145206e-06 1.62834003e-06 1.61519245e-06 1.81694650e-06
 3.22474052e-06 1.61519245e-06]


In [ ]:
%%time
r =  np.ones((n,1))
error = 1
with pymp.Parallel(4) as p:

    for _ in p.range(50):

        r_new = A @ r
        r_new += np.sum(r) * tele_p / n

        error = np.sum(np.absolute(r - r_new))
        r = r_new
        if error < 1e-6:
            break

    r /= np.sum(r)
    r = r.reshape(n)

    print("part of my result\n", r[:50])

part of my result
part of my result
part of my result
  [3.81744014e-05 3.96893451e-05 9.08553224e-06 4.02198517e-05
 3.88551842e-05 7.51398079e-06 1.13312208e-05 3.54385345e-06
 4.13151742e-06 4.13151742e-06 5.07782472e-06 2.85834932e-06
 2.98523312e-06 3.17950951e-06 8.44377842e-06 3.52849595e-06
 1.07688616e-06 1.45592772e-04 3.39753909e-04 7.80484605e-06
 7.66762886e-06 1.08332312e-06 5.15304913e-06 4.81523940e-06
 3.76235344e-06 7.05585124e-06 3.99966327e-06 3.49354376e-06
 1.56504770e-05 3.73238429e-06 3.70224337e-06 3.70224337e-06
 3.70224337e-06 4.70868168e-06 2.37020085e-06 9.98019379e-06
 1.25822327e-05 3.41127512e-04 9.60382525e-06 9.47767356e-06
 9.76265829e-06 9.84493237e-06 9.81898785e-06 9.65778346e-06
 9.70758877e-06 1.93364728e-06 1.92883275e-06 2.06032793e-06
 5.26402749e-06 1.92883275e-06] 
[3.81744014e-05 3.96893451e-05 9.08553224e-06 4.02198517e-05
 3.88551842e-05 7.51398079e-06 1.13312208e-05 3.54385345e-06
 4.13151742e-06 4.13151742e-06 5.07782472e-06 2.85834932e

In [ ]:
!pip install pymp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.7 MB 39.3 MB/s 
     |████████████████████████████████| 17.3 MB 20.2 MB/s 
     |████████████████████████████████| 18.5 MB 164 kB/s 
  Created wheel for pymp: filename=pymp-0.0.6-py3-none-any.whl size=10732656 sha256=db2432935a723c92bfd74753f804ca38ecef9b7f9832f5cccb79044c6e1c2df5
  Stored in directory: /root/.cache/pip/wheels/e3/c0/f4/b490b163052ba2baa955f8a177e611b72cdd9a3028facfdc86
Successfully built pymp
  Attempting uninstall: bokeh
    Found existing installation: bokeh 2.3.3
    Uninstalling bokeh-2.3.3:
      Successfully uninstalled bokeh-2.3.3
  Attempting uninstall: panel
    Found existing installation: panel 0.12.1
    Uninstalling panel-0.12.1:
      Successfully uninstalled panel-0.12.1


In [ ]:
r =  np.ones((n,1))
error = 1
for _ in range(100):
#     if _ % 100 == 0:
#         print("iter", _)
    r_new = A @ r
    r_new += np.sum(r) * tele_p / n
#     r_new /= np.sum(r_new)
    error = np.sum(np.absolute(r - r_new))
    r = r_new
    if error < 1e-6:
        break

r /= np.sum(r)
r = r.reshape(n)

print("part of my result\n", r[:50])

part of my result
 [3.64631200e-05 3.79148977e-05 8.65254646e-06 3.84222598e-05
 3.71291484e-05 7.18235005e-06 1.08737820e-05 3.38745041e-06
 3.93542327e-06 3.93542327e-06 4.89030713e-06 2.72347681e-06
 2.84317175e-06 3.02949457e-06 8.03802726e-06 3.36082553e-06
 1.03333572e-06 1.40538665e-04 3.27574613e-04 7.47987236e-06
 7.28727277e-06 1.03948413e-06 4.90779352e-06 4.58973203e-06
 3.58364975e-06 6.74176067e-06 3.80755620e-06 3.32570975e-06
 1.49012665e-05 3.55625401e-06 3.52576121e-06 3.52576121e-06
 3.52576121e-06 4.51944096e-06 2.30479511e-06 9.70688504e-06
 1.21980914e-05 3.30972264e-04 9.34149806e-06 9.21901946e-06
 9.49834322e-06 9.57868071e-06 9.54882022e-06 9.39388398e-06
 9.44438702e-06 1.88003632e-06 1.87636568e-06 2.00273775e-06
 5.09777845e-06 1.87636568e-06]


In [ ]:
import pymp

In [ ]:
from __future__ import print_function

In [ ]:
#pip install --upgrade pymp-pypi

In [ ]:

r = np.ones((n,1))
error = 1
with pymp.Parallel(4) as p:
  for _ in range(100):
#     if _ % 100 == 0:
#         print("iter", _)
    r_new = A @ r
    r_new += np.sum(r) * tele_p / n
#     r_new /= np.sum(r_new)
    error = np.sum(np.absolute(r - r_new))
    r = r_new
    if error < 1e-6:
        break

r /= np.sum(r)
r = r.reshape(n)

print("part of my result\n", r[:50])

part of my result
 [3.64631200e-05 3.79148977e-05 8.65254646e-06 3.84222598e-05
 3.71291484e-05 7.18235005e-06 1.08737820e-05 3.38745041e-06
 3.93542327e-06 3.93542327e-06 4.89030713e-06 2.72347681e-06
 2.84317175e-06 3.02949457e-06 8.03802726e-06 3.36082553e-06
 1.03333572e-06 1.40538665e-04 3.27574613e-04 7.47987236e-06
 7.28727277e-06 1.03948413e-06 4.90779352e-06 4.58973203e-06
 3.58364975e-06 6.74176067e-06 3.80755620e-06 3.32570975e-06
 1.49012665e-05 3.55625401e-06 3.52576121e-06 3.52576121e-06
 3.52576121e-06 4.51944096e-06 2.30479511e-06 9.70688504e-06
 1.21980914e-05 3.30972264e-04 9.34149806e-06 9.21901946e-06
 9.49834322e-06 9.57868071e-06 9.54882022e-06 9.39388398e-06
 9.44438702e-06 1.88003632e-06 1.87636568e-06 2.00273775e-06
 5.09777845e-06 1.87636568e-06]


In [ ]:
m=max(r)

In [ ]:
t=min(r)

In [ ]:
k=sum(r)/len(r)

In [ ]:
np.where(r==m)

(array([2135]),)

In [ ]:
np.where(r==t)

(array([   249,    431,    633, ..., 855799, 855800, 855801]),)

In [ ]:
np.where(r==k)

(array([], dtype=int64),)